# **Introduction:**

This file serves to compare the inference systems that had been developed.

**Date Created: 16/2/2025**

**Date Modified: 18/2/2025**

# **Import Packages:**

This section imports the necessary packages.

In [18]:
# import packages:
import numpy as np
import time
import os
from keras.models import load_model
import tensorflow as tf
from tensorflow.keras.losses import MeanSquaredError
from sklearn.preprocessing import StandardScaler
from pickle import load
from ANFIS_Custom_Layers import *
from PythonFISFunctionV3 import *

# **Create FIS:**

This section creates the rule-base for the FIS.

In [19]:
rulebase = fis_create()

# **Load the ANN Model:**

This section loads the ANN model and the scaler that is used for the ANN.

In [20]:
# define the ANN directory path:
ann_path = os.path.join(os.getcwd(), 'ANN_Model')

# load the ann model:
ann_model = load_model(ann_path + '/ann_model.h5', custom_objects = {'mse' : MeanSquaredError()})

# load the ann scaler:
ann_scaler = load(open(ann_path + '/ann_scaler.pkl', 'rb'))

# need to warm the model to prevent retracing of computational graph on first inference:
batch_size = 4
dummy_input = np.zeros((batch_size, 3), dtype = np.float32)
dummy_input = ann_scaler.transform(dummy_input)
ann_model.predict(dummy_input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


c:\Users\mtidd2\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[9.33442],
       [9.33442],
       [9.33442],
       [9.33442]], dtype=float32)

# **Load the ANFIS Model:**

This section loads the ANFIS model and the scaler.

In [21]:
# define the ANFIS directory path:
anfis_path = os.path.join(os.getcwd(), 'ANFIS_Model')

# define the dictionary of custom objects:
custom_objects = {
    # # layers:
    'MF_Layer'          : MF_Layer,
    'FS_Layer'          : FS_Layer,
    'NM_Layer'          : NM_Layer,
    'CN_Layer'          : CN_Layer,
    'O_Layer'           : O_Layer,

    # other:
    'OrderedConstraint' : OrderedConstraint(),
    'mse'               : MeanSquaredError()
}

# load the model:
anfis_model = load_model(anfis_path + '/anfis_model.h5', custom_objects = custom_objects)
anfis_model.compile(optimizer="adam", loss="mse") 

# load the scaler:
anfis_scaler = load(open(anfis_path + '/anfis_scaler.pkl', 'rb'))

# warmup the model:
batch_size = 4  
dummy_input = np.zeros((batch_size, 3), dtype=np.float32)  
dummy_input = anfis_scaler.transform(dummy_input)
anfis_model.predict(dummy_input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


array([[9.028122],
       [9.028122],
       [9.028122],
       [9.028122]], dtype=float32)

# **Test the Models:**

This section tests the models against one another on the same input dataset

In [22]:
# define the input values:
input_data = np.array([[2, 14.024, 34.047],
                      [0, 35.213, 0], 
                      [1, 15.374, 18.783], 
                      [1, 14.620, 11.881]])
